# Module 06: Library Visualizations and Statistics

**Difficulty**: ⭐⭐
**Estimated Time**: 30 minutes
**Prerequisites**: [Module 01: Metadata Management](01_metadata_management.ipynb), [Module 02: Audio Analysis](02_audio_analysis.ipynb)

## Learning Objectives
By the end of this notebook, you will be able to:
1. Create visualizations of your music library statistics
2. Generate charts showing format, genre, and quality distributions
3. Visualize your collection timeline and growth
4. Create interactive dashboards for exploration
5. Export reports and statistics
6. Identify trends and patterns in your collection

## Overview
Transform raw statistics into visual insights! This module creates beautiful charts and graphs to help you understand your music collection:

- **Distribution Charts**: See your music by format, quality, genre, artist
- **Timeline Analysis**: Track when you added music over time
- **Quality Insights**: Visualize audio quality across your library
- **Interactive Exploration**: Drill down into specific categories

### Why Visualizations Matter
- **Quick Insights**: Spot patterns at a glance
- **Library Health**: Identify quality issues or gaps
- **Decision Making**: Guide what to download or upgrade
- **Progress Tracking**: See your collection grow over time

## 1. Setup and Configuration

In [ ]:
# Import required libraries
import os
from pathlib import Path
from collections import Counter, defaultdict
from typing import List, Dict, Optional
from datetime import datetime
import warnings

# Data handling
import pandas as pd
import numpy as np

# Visualization libraries
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.patches import Patch
import seaborn as sns

# Interactive visualizations (optional)
try:
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    PLOTLY_AVAILABLE = True
except ImportError:
    PLOTLY_AVAILABLE = False
    print("⚠️ Plotly not installed. Install with: pip install plotly")
    print("   Interactive visualizations will not be available.")

# For better display
from IPython.display import display, HTML

# Audio analysis imports
from mutagen import File

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)

# Matplotlib style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Figure size defaults
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

print("✓ Libraries imported successfully")
print(f"  Matplotlib: Available")
print(f"  Seaborn: Available")
print(f"  Plotly: {'Available' if PLOTLY_AVAILABLE else 'Not available'}")

In [ ]:
# Configure paths
MUSIC_LIBRARY_PATH = Path('../../music')
AUDIO_EXTENSIONS = {'.mp3', '.flac', '.wav', '.m4a', '.aac', '.ogg', '.wma', '.opus'}

# Quality tier colors for consistency
QUALITY_COLORS = {
    'lossless': '#2ecc71',  # Green
    'high': '#3498db',      # Blue
    'medium': '#f39c12',    # Orange
    'low': '#e67e22',       # Dark orange
    'poor': '#e74c3c',      # Red
    'unknown': '#95a5a6'    # Gray
}

if not MUSIC_LIBRARY_PATH.exists():
    MUSIC_LIBRARY_PATH.mkdir(parents=True, exist_ok=True)
    print(f"Created: {MUSIC_LIBRARY_PATH.absolute()}")
else:
    print(f"✓ Music library path: {MUSIC_LIBRARY_PATH.absolute()}")

## 2. Data Collection Functions

In [ ]:
# Import helper functions from previous modules
# (In a production environment, these would be in a shared module)

def extract_metadata(file_path: Path) -> Dict:
    """Extract metadata from audio file."""
    metadata = {'artist': None, 'album': None, 'title': None, 'date': None, 'genre': None}
    try:
        audio = File(file_path, easy=True)
        if audio:
            for tag in metadata.keys():
                if tag in audio and audio[tag]:
                    metadata[tag] = str(audio[tag][0]) if isinstance(audio[tag], list) else str(audio[tag])
    except:
        pass
    return metadata

def get_audio_properties(file_path: Path) -> Dict:
    """Extract audio properties."""
    props = {'bitrate': None, 'sample_rate': None, 'duration': None, 'channels': None}
    try:
        audio = File(file_path)
        if audio and audio.info:
            if hasattr(audio.info, 'bitrate'):
                props['bitrate'] = round(audio.info.bitrate / 1000, 0)
            if hasattr(audio.info, 'sample_rate'):
                props['sample_rate'] = audio.info.sample_rate
            if hasattr(audio.info, 'length'):
                props['duration'] = round(audio.info.length, 2)
            if hasattr(audio.info, 'channels'):
                props['channels'] = audio.info.channels
    except:
        pass
    return props

def categorize_quality(file_path: Path, bitrate: Optional[float] = None) -> str:
    """Categorize audio quality."""
    lossless = {'.flac', '.wav', '.aiff', '.ape', '.alac'}
    if file_path.suffix.lower() in lossless:
        return 'lossless'
    if bitrate is None:
        props = get_audio_properties(file_path)
        bitrate = props.get('bitrate')
    if bitrate is None:
        return 'unknown'
    if bitrate >= 256:
        return 'high'
    elif bitrate >= 192:
        return 'medium'
    elif bitrate >= 128:
        return 'low'
    else:
        return 'poor'

def scan_library_full(library_path: Path) -> List[Dict]:
    """Complete library scan with all metadata and properties."""
    files = []
    print(f"Scanning library: {library_path}")
    
    count = 0
    for root, dirs, filenames in os.walk(library_path):
        for filename in filenames:
            file_path = Path(root) / filename
            if file_path.suffix.lower() in AUDIO_EXTENSIONS:
                count += 1
                if count % 50 == 0:
                    print(f"  Scanned {count} files...")
                
                stat = file_path.stat()
                metadata = extract_metadata(file_path)
                audio_props = get_audio_properties(file_path)
                
                files.append({
                    'filename': filename,
                    'path': str(file_path),
                    'extension': file_path.suffix.lower(),
                    'size_mb': stat.st_size / (1024 * 1024),
                    'modified': datetime.fromtimestamp(stat.st_mtime),
                    'artist': metadata.get('artist'),
                    'album': metadata.get('album'),
                    'title': metadata.get('title'),
                    'date': metadata.get('date'),
                    'genre': metadata.get('genre'),
                    'bitrate': audio_props.get('bitrate'),
                    'duration': audio_props.get('duration'),
                    'quality_tier': categorize_quality(file_path, audio_props.get('bitrate'))
                })
    
    print(f"✓ Scan complete: {count} files")
    return files

print("✓ Data collection functions loaded")

## 3. Static Visualization Functions (Matplotlib/Seaborn)

In [ ]:
def plot_format_distribution(music_files: List[Dict], figsize=(10, 6)):
    """
    Plot distribution of audio formats (pie chart).
    """
    formats = Counter(f['extension'] for f in music_files)
    
    fig, ax = plt.subplots(figsize=figsize)
    
    labels = [fmt.upper().replace('.', '') for fmt in formats.keys()]
    sizes = list(formats.values())
    
    ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
    ax.set_title(f'Audio Format Distribution (Total: {len(music_files)} files)', 
                 fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()


def plot_quality_distribution(music_files: List[Dict], figsize=(12, 6)):
    """
    Plot quality tier distribution (bar chart).
    """
    quality_counts = Counter(f.get('quality_tier', 'unknown') for f in music_files)
    
    # Order by quality
    quality_order = ['lossless', 'high', 'medium', 'low', 'poor', 'unknown']
    ordered_data = [(q, quality_counts.get(q, 0)) for q in quality_order if quality_counts.get(q, 0) > 0]
    
    labels, values = zip(*ordered_data) if ordered_data else ([], [])
    colors = [QUALITY_COLORS.get(q, '#95a5a6') for q in labels]
    
    fig, ax = plt.subplots(figsize=figsize)
    bars = ax.bar(range(len(labels)), values, color=colors)
    
    ax.set_xticks(range(len(labels)))
    ax.set_xticklabels([l.capitalize() for l in labels])
    ax.set_ylabel('Number of Files', fontsize=12)
    ax.set_title('Audio Quality Distribution', fontsize=14, fontweight='bold')
    ax.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bar, value in zip(bars, values):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(value)}\n({value/len(music_files)*100:.1f}%)',
                ha='center', va='bottom', fontsize=10)
    
    plt.tight_layout()
    plt.show()


def plot_genre_distribution(music_files: List[Dict], top_n=10, figsize=(12, 6)):
    """
    Plot top N genres (horizontal bar chart).
    """
    genres = [f.get('genre') for f in music_files if f.get('genre')]
    
    if not genres:
        print("No genre information available in files.")
        return
    
    genre_counts = Counter(genres)
    top_genres = genre_counts.most_common(top_n)
    
    labels, values = zip(*top_genres)
    
    fig, ax = plt.subplots(figsize=figsize)
    y_pos = np.arange(len(labels))
    
    bars = ax.barh(y_pos, values, color=sns.color_palette("husl", len(labels)))
    ax.set_yticks(y_pos)
    ax.set_yticklabels(labels)
    ax.invert_yaxis()
    ax.set_xlabel('Number of Songs', fontsize=12)
    ax.set_title(f'Top {top_n} Genres', fontsize=14, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    
    # Add value labels
    for bar, value in zip(bars, values):
        width = bar.get_width()
        ax.text(width, bar.get_y() + bar.get_height()/2.,
                f' {int(value)}',
                ha='left', va='center', fontsize=10)
    
    plt.tight_layout()
    plt.show()


def plot_top_artists(music_files: List[Dict], top_n=15, figsize=(12, 8)):
    """
    Plot top N artists by song count.
    """
    artists = [f.get('artist') for f in music_files if f.get('artist')]
    
    if not artists:
        print("No artist information available.")
        return
    
    artist_counts = Counter(artists)
    top_artists = artist_counts.most_common(top_n)
    
    labels, values = zip(*top_artists)
    
    fig, ax = plt.subplots(figsize=figsize)
    y_pos = np.arange(len(labels))
    
    bars = ax.barh(y_pos, values, color=sns.color_palette("viridis", len(labels)))
    ax.set_yticks(y_pos)
    ax.set_yticklabels(labels)
    ax.invert_yaxis()
    ax.set_xlabel('Number of Songs', fontsize=12)
    ax.set_title(f'Top {top_n} Artists', fontsize=14, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    
    for bar, value in zip(bars, values):
        width = bar.get_width()
        ax.text(width, bar.get_y() + bar.get_height()/2.,
                f' {int(value)}',
                ha='left', va='center', fontsize=10)
    
    plt.tight_layout()
    plt.show()


def plot_bitrate_histogram(music_files: List[Dict], figsize=(12, 6)):
    """
    Plot bitrate distribution histogram (excluding lossless).
    """
    bitrates = [f['bitrate'] for f in music_files 
                if f.get('bitrate') and f.get('quality_tier') != 'lossless']
    
    if not bitrates:
        print("No bitrate information available.")
        return
    
    fig, ax = plt.subplots(figsize=figsize)
    
    n, bins, patches = ax.hist(bitrates, bins=30, edgecolor='black', alpha=0.7)
    
    # Color bars by quality tier
    for patch, left_edge in zip(patches, bins[:-1]):
        if left_edge >= 256:
            patch.set_facecolor(QUALITY_COLORS['high'])
        elif left_edge >= 192:
            patch.set_facecolor(QUALITY_COLORS['medium'])
        elif left_edge >= 128:
            patch.set_facecolor(QUALITY_COLORS['low'])
        else:
            patch.set_facecolor(QUALITY_COLORS['poor'])
    
    ax.set_xlabel('Bitrate (kbps)', fontsize=12)
    ax.set_ylabel('Number of Files', fontsize=12)
    ax.set_title('Bitrate Distribution (Lossy Files)', fontsize=14, fontweight='bold')
    ax.grid(axis='y', alpha=0.3)
    
    # Add average line
    avg_bitrate = np.mean(bitrates)
    ax.axvline(avg_bitrate, color='red', linestyle='--', linewidth=2, 
               label=f'Average: {avg_bitrate:.0f} kbps')
    ax.legend()
    
    plt.tight_layout()
    plt.show()


def plot_library_timeline(music_files: List[Dict], figsize=(14, 6)):
    """
    Plot library growth over time (by file modification date).
    """
    dates = [f['modified'] for f in music_files if f.get('modified')]
    
    if not dates:
        print("No date information available.")
        return
    
    # Sort dates
    dates.sort()
    
    # Create cumulative count
    cumulative = list(range(1, len(dates) + 1))
    
    fig, ax = plt.subplots(figsize=figsize)
    
    ax.plot(dates, cumulative, linewidth=2, color='#3498db')
    ax.fill_between(dates, cumulative, alpha=0.3, color='#3498db')
    
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Total Files', fontsize=12)
    ax.set_title('Library Growth Over Time', fontsize=14, fontweight='bold')
    ax.grid(alpha=0.3)
    
    # Format x-axis
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()

print("✓ Static visualization functions loaded")

## 4. Interactive Visualizations (Plotly)

In [ ]:
def create_interactive_dashboard(music_files: List[Dict]):
    """
    Create an interactive dashboard with multiple charts.
    Requires plotly.
    """
    if not PLOTLY_AVAILABLE:
        print("Plotly not available. Install with: pip install plotly")
        return
    
    # Prepare data
    df = pd.DataFrame(music_files)
    
    # Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Format Distribution', 'Quality Distribution',
                        'Top 10 Artists', 'Bitrate Distribution'),
        specs=[[{'type': 'pie'}, {'type': 'bar'}],
               [{'type': 'bar'}, {'type': 'histogram'}]]
    )
    
    # 1. Format Distribution (Pie)
    format_counts = df['extension'].value_counts()
    fig.add_trace(
        go.Pie(labels=[e.upper().replace('.', '') for e in format_counts.index],
               values=format_counts.values,
               name='Format'),
        row=1, col=1
    )
    
    # 2. Quality Distribution (Bar)
    quality_counts = df['quality_tier'].value_counts()
    quality_order = ['lossless', 'high', 'medium', 'low', 'poor', 'unknown']
    quality_counts = quality_counts.reindex(quality_order, fill_value=0)
    colors = [QUALITY_COLORS.get(q, '#95a5a6') for q in quality_counts.index]
    
    fig.add_trace(
        go.Bar(x=[q.capitalize() for q in quality_counts.index],
               y=quality_counts.values,
               marker_color=colors,
               name='Quality'),
        row=1, col=2
    )
    
    # 3. Top 10 Artists (Bar)
    if 'artist' in df.columns:
        artist_counts = df['artist'].value_counts().head(10)
        fig.add_trace(
            go.Bar(y=artist_counts.index,
                   x=artist_counts.values,
                   orientation='h',
                   name='Artists'),
            row=2, col=1
        )
    
    # 4. Bitrate Distribution (Histogram)
    lossy_bitrates = df[df['quality_tier'] != 'lossless']['bitrate'].dropna()
    if len(lossy_bitrates) > 0:
        fig.add_trace(
            go.Histogram(x=lossy_bitrates,
                        nbinsx=30,
                        name='Bitrate'),
            row=2, col=2
        )
    
    # Update layout
    fig.update_layout(
        title_text="Music Library Dashboard",
        title_font_size=20,
        showlegend=False,
        height=800
    )
    
    fig.show()


def create_quality_sunburst(music_files: List[Dict]):
    """
    Create an interactive sunburst chart showing format → quality hierarchy.
    """
    if not PLOTLY_AVAILABLE:
        print("Plotly not available.")
        return
    
    df = pd.DataFrame(music_files)
    
    # Create hierarchy data
    hierarchy_data = []
    for _, row in df.iterrows():
        hierarchy_data.append({
            'format': row['extension'].upper().replace('.', ''),
            'quality': row.get('quality_tier', 'unknown').capitalize(),
            'count': 1
        })
    
    hierarchy_df = pd.DataFrame(hierarchy_data)
    grouped = hierarchy_df.groupby(['format', 'quality']).sum().reset_index()
    
    fig = px.sunburst(
        grouped,
        path=['format', 'quality'],
        values='count',
        title='Music Library: Format → Quality Distribution',
        color='quality',
        color_discrete_map={
            'Lossless': QUALITY_COLORS['lossless'],
            'High': QUALITY_COLORS['high'],
            'Medium': QUALITY_COLORS['medium'],
            'Low': QUALITY_COLORS['low'],
            'Poor': QUALITY_COLORS['poor'],
            'Unknown': QUALITY_COLORS['unknown']
        }
    )
    
    fig.update_layout(height=600)
    fig.show()

print("✓ Interactive visualization functions loaded")

## 5. Report Generation

In [ ]:
def generate_text_report(music_files: List[Dict]) -> str:
    """
    Generate a comprehensive text report.
    """
    df = pd.DataFrame(music_files)
    
    lines = []
    lines.append("="*80)
    lines.append("MUSIC LIBRARY REPORT")
    lines.append(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    lines.append("="*80)
    lines.append("")
    
    # Overview
    lines.append("OVERVIEW")
    lines.append("-"*80)
    lines.append(f"Total Files: {len(df)}")
    total_size_gb = df['size_mb'].sum() / 1024
    lines.append(f"Total Size: {total_size_gb:.2f} GB")
    total_duration_hours = df['duration'].sum() / 3600 if 'duration' in df.columns else 0
    lines.append(f"Total Duration: {total_duration_hours:.2f} hours")
    lines.append("")
    
    # Format Distribution
    lines.append("FORMAT DISTRIBUTION")
    lines.append("-"*80)
    format_counts = df['extension'].value_counts()
    for fmt, count in format_counts.items():
        pct = (count / len(df)) * 100
        lines.append(f"  {fmt.upper().replace('.', ''):6s} : {count:5d} files ({pct:5.1f}%)")
    lines.append("")
    
    # Quality Distribution
    lines.append("QUALITY DISTRIBUTION")
    lines.append("-"*80)
    quality_counts = df['quality_tier'].value_counts()
    for quality, count in quality_counts.items():
        pct = (count / len(df)) * 100
        lines.append(f"  {quality.capitalize():10s} : {count:5d} files ({pct:5.1f}%)")
    lines.append("")
    
    # Bitrate Statistics
    lossy = df[df['quality_tier'] != 'lossless']['bitrate'].dropna()
    if len(lossy) > 0:
        lines.append("BITRATE STATISTICS (Lossy Files)")
        lines.append("-"*80)
        lines.append(f"  Average: {lossy.mean():.0f} kbps")
        lines.append(f"  Median:  {lossy.median():.0f} kbps")
        lines.append(f"  Min:     {lossy.min():.0f} kbps")
        lines.append(f"  Max:     {lossy.max():.0f} kbps")
        lines.append("")
    
    # Top Artists
    if 'artist' in df.columns:
        artist_counts = df['artist'].value_counts().head(10)
        if len(artist_counts) > 0:
            lines.append("TOP 10 ARTISTS")
            lines.append("-"*80)
            for i, (artist, count) in enumerate(artist_counts.items(), 1):
                lines.append(f"  {i:2d}. {artist:40s} : {count:4d} songs")
            lines.append("")
    
    # Top Genres
    if 'genre' in df.columns:
        genre_counts = df['genre'].value_counts().head(10)
        if len(genre_counts) > 0:
            lines.append("TOP 10 GENRES")
            lines.append("-"*80)
            for i, (genre, count) in enumerate(genre_counts.items(), 1):
                lines.append(f"  {i:2d}. {genre:30s} : {count:4d} songs")
            lines.append("")
    
    lines.append("="*80)
    
    return "\n".join(lines)


def export_report_to_file(music_files: List[Dict], output_path: Path):
    """
    Export report to text file.
    """
    report = generate_text_report(music_files)
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(report)
    print(f"✓ Report saved to: {output_path}")


def export_stats_to_csv(music_files: List[Dict], output_path: Path):
    """
    Export detailed statistics to CSV.
    """
    df = pd.DataFrame(music_files)
    df.to_csv(output_path, index=False)
    print(f"✓ Statistics exported to: {output_path}")

print("✓ Report generation functions loaded")

---
## 6. Usage Examples

### 6.1 Scan Library

In [ ]:
# Scan library with full metadata
print("Scanning library...\n")
all_music_files = scan_library_full(MUSIC_LIBRARY_PATH)

print(f"\n✓ Found {len(all_music_files)} files")

if all_music_files:
    print(f"  Ready to visualize!")
else:
    print("\nNo music files found. Add some music to see visualizations!")

### 6.2 Format Distribution

In [ ]:
# Plot format distribution
if all_music_files:
    plot_format_distribution(all_music_files)
else:
    print("No files to visualize.")

### 6.3 Quality Distribution

In [ ]:
# Plot quality tier distribution
if all_music_files:
    plot_quality_distribution(all_music_files)
else:
    print("No files to visualize.")

### 6.4 Genre Distribution

In [ ]:
# Plot top genres
if all_music_files:
    plot_genre_distribution(all_music_files, top_n=10)
else:
    print("No files to visualize.")

### 6.5 Top Artists

In [ ]:
# Plot top artists
if all_music_files:
    plot_top_artists(all_music_files, top_n=15)
else:
    print("No files to visualize.")

### 6.6 Bitrate Distribution

In [ ]:
# Plot bitrate histogram
if all_music_files:
    plot_bitrate_histogram(all_music_files)
else:
    print("No files to visualize.")

### 6.7 Library Growth Timeline

In [ ]:
# Plot library growth over time
if all_music_files:
    plot_library_timeline(all_music_files)
else:
    print("No files to visualize.")

### 6.8 Interactive Dashboard (Plotly)

In [ ]:
# Create interactive dashboard
if all_music_files and PLOTLY_AVAILABLE:
    create_interactive_dashboard(all_music_files)
elif not PLOTLY_AVAILABLE:
    print("Install plotly for interactive visualizations: pip install plotly")
else:
    print("No files to visualize.")

### 6.9 Quality Sunburst Chart

In [ ]:
# Create sunburst chart
if all_music_files and PLOTLY_AVAILABLE:
    create_quality_sunburst(all_music_files)
elif not PLOTLY_AVAILABLE:
    print("Install plotly for interactive visualizations: pip install plotly")
else:
    print("No files to visualize.")

### 6.10 Text Report

In [ ]:
# Generate and display text report
if all_music_files:
    report = generate_text_report(all_music_files)
    print(report)
else:
    print("No files to report on.")

### 6.11 Export Reports

In [ ]:
# Export report to file
if all_music_files:
    # Uncomment to export:
    # report_path = MUSIC_LIBRARY_PATH / 'library_report.txt'
    # export_report_to_file(all_music_files, report_path)
    
    # csv_path = MUSIC_LIBRARY_PATH / 'library_stats.csv'
    # export_stats_to_csv(all_music_files, csv_path)
    
    print("💡 Uncomment the code above to export reports")
else:
    print("No files to export.")

## 7. Summary

### What We've Learned

In this module, we've created powerful visualizations for library analysis:

1. **Static Charts** (Matplotlib/Seaborn):
   - Format distribution pie charts
   - Quality tier bar charts
   - Genre and artist rankings
   - Bitrate histograms
   - Library growth timelines

2. **Interactive Visualizations** (Plotly):
   - Multi-panel dashboards
   - Sunburst hierarchy charts
   - Drill-down exploration

3. **Reports and Exports**:
   - Comprehensive text reports
   - CSV data exports
   - Shareable statistics

### Key Functions

**Static Visualizations:**
- `plot_format_distribution()` - Pie chart of formats
- `plot_quality_distribution()` - Bar chart of quality tiers
- `plot_genre_distribution()` - Top genres
- `plot_top_artists()` - Most common artists
- `plot_bitrate_histogram()` - Bitrate distribution
- `plot_library_timeline()` - Growth over time

**Interactive:**
- `create_interactive_dashboard()` - Multi-panel dashboard
- `create_quality_sunburst()` - Hierarchical visualization

**Reports:**
- `generate_text_report()` - Comprehensive text report
- `export_report_to_file()` - Save to .txt
- `export_stats_to_csv()` - Export data to CSV

### Best Practices

1. **Regular Analysis**: Run visualizations periodically to track library health
2. **Identify Trends**: Look for patterns in quality or format distribution
3. **Spot Issues**: Low quality concentrations indicate upgrade opportunities
4. **Track Growth**: Timeline shows when you actively added music
5. **Export Reports**: Share statistics or track changes over time

### Insights to Look For

**Quality Analysis:**
- High percentage of poor/low quality → Consider re-downloading
- Mixed quality from same artist → Inconsistent sources
- All high/lossless → Well-curated collection

**Format Analysis:**
- Dominated by MP3 → Common, compatible
- Mix of FLAC/MP3 → Archival + portable strategy
- Obscure formats → Consider converting for compatibility

**Artist/Genre:**
- Top artists → Your listening preferences
- Genre diversity → Eclectic vs. focused collection
- Outliers → One-hit wonders or exploration

### Next Steps

Use visualizations to:
- **Identify upgrades** → Module 02 to find low-quality files
- **Plan organization** → Module 04 to structure by genre/artist
- **Create playlists** → Module 07 based on insights
- **Track progress** → Re-run after improvements

### Additional Resources

- [Matplotlib Gallery](https://matplotlib.org/stable/gallery/index.html)
- [Seaborn Tutorial](https://seaborn.pydata.org/tutorial.html)
- [Plotly Examples](https://plotly.com/python/)
- [Data Visualization Best Practices](https://www.interaction-design.org/literature/article/data-visualization-for-human-perception)